# Импорт необходимых библиотек

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import folium.plugins
from folium import LayerControl, FeatureGroup
from pyproj import Transformer
from shapely.geometry import Point
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Суть задания:

Необходимо прочитать .csv "test.csv" в котором записаны геоданные (информация в заголовках) в системе координат `source` (нужно будет с помощью pyproj перевести в `target` для отображения на карте с помощью библиотеки folium). Предвартельно нужно будет преобразовать df в формат geopandas.

Нужно выяснить сколько было различных проездов автомобиля (прерывания более 0.5 секунды), визуализировать эти маршруты, визуализировать графики скорости проездов.

In [2]:
source = "EPSG:32637"
target = "EPSG:4326"

In [3]:
df = gpd.read_file('test-4.csv')
df.head()

,field_1,panoram stream number,frame name,GPS time,X,Y,Z,roll,pitch,heading,date time,geometry
0,0,36,ladybug_panoramic_000000,372238.374093,399070.538384,6186248.863528,177.289079,178.355788,178.465146,13.182995,2020-11-12 07:23:58,None
1,1,36,ladybug_panoramic_000001,372238.874115,399072.484287,6186257.807541,177.252496,178.469506,178.115046,13.246203,2020-11-12 07:23:58,None
2,2,36,ladybug_panoramic_000002,372239.374138,399074.43753,6186266.884536,177.218097,178.502584,178.319568,13.085446,2020-11-12 07:23:59,None
3,3,36,ladybug_panoramic_000003,372239.87416,399076.386642,6186276.058376,177.172916,178.456564,177.944437,12.959536,2020-11-12 07:23:59,None
4,4,36,ladybug_panoramic_000004,372240.374182,399078.315928,6186285.339778,177.127745,178.859275,178.618339,12.458477,2020-11-12 07:24:00,None


In [4]:
transformer = Transformer.from_crs(source,target)

In [5]:
df.X,df.Y,df.Z, df['GPS time']  = transformer.transform(df.X,df.Y,df.Z, df['GPS time'])

# Преобразуем DataFrame в geopandas

In [6]:
gdf = gpd.GeoDataFrame(
    df, geometry = gpd.points_from_xy(df.X,df.Y,df.Z, crs=target ))

In [7]:
gdf.head()

,field_1,panoram stream number,frame name,GPS time,X,Y,Z,roll,pitch,heading,date time,geometry
0,0,36,ladybug_panoramic_000000,372238.374093,55.811275,37.389467,177.289079,178.355788,178.465146,13.182995,2020-11-12 07:23:58,POINT Z (55.81128 37.38947 177.28908)
1,1,36,ladybug_panoramic_000001,372238.874115,55.811356,37.389495,177.252496,178.469506,178.115046,13.246203,2020-11-12 07:23:58,POINT Z (55.81136 37.38949 177.25250)
2,2,36,ladybug_panoramic_000002,372239.374138,55.811438,37.389522,177.218097,178.502584,178.319568,13.085446,2020-11-12 07:23:59,POINT Z (55.81144 37.38952 177.21810)
3,3,36,ladybug_panoramic_000003,372239.874160,55.811521,37.389550,177.172916,178.456564,177.944437,12.959536,2020-11-12 07:23:59,POINT Z (55.81152 37.38955 177.17292)
4,4,36,ladybug_panoramic_000004,372240.374182,55.811604,37.389578,177.127745,178.859275,178.618339,12.458477,2020-11-12 07:24:00,POINT Z (55.81160 37.38958 177.12775)


# Рисуем карту с помощью folium

In [8]:
#colots for routes

colors = ['#667fce', '#e3d990', '#30a0c8', '#342d04', '#86f181', '#be496c', '#ae360a', '#efd8c2',
          '#7fbda6', '#1d1417', '#f3f80f']

In [9]:
routing_map = folium.Map(location=[np.mean(gdf['X'].values), np.mean(gdf['Y'].values)], zoom_start=12)

#Adds a fullscreen button to your map.
folium.plugins.Fullscreen(
    position="topright",
    title="Expand",
    title_cancel="Exit",
    force_separate_button=True,
).add_to(routing_map)

routing_map

In [10]:
gdf['TimeDiff'] = gdf['GPS time'].diff().fillna(0)
gdf['PointsDiff'] = np.sqrt(gdf['X'].diff()**2 + gdf['Y'].diff()**2).fillna(0)
gdf['Distance'] = gdf['geometry'].distance(gdf['geometry'].shift()).fillna(0)

curves = []
speed = []
for i, number in enumerate(gdf['panoram stream number'].unique()):
    curve = df.loc[gdf['panoram stream number'] == number]
    curve['Speed'] = ((curve[['X','Y','Z']].diff() ** 2).sum(axis=1)) ** 0.5 / curve['GPS time'].diff() * 3.6
    curves.append(curve)
curves[0]

<ipython-input-10-734e9c7ce655>:3: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['Distance'] = gdf['geometry'].distance(gdf['geometry'].shift()).fillna(0)
/Users/nikitapronin/opt/anaconda3/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


,field_1,panoram stream number,frame name,GPS time,X,Y,Z,roll,pitch,heading,date time,geometry,TimeDiff,PointsDiff,Distance,Speed
0,0,36,ladybug_panoramic_000000,372238.374093,55.811275,37.389467,177.289079,178.355788,178.465146,13.182995,2020-11-12 07:23:58,POINT Z (55.81128 37.38947 177.28908),0.000000,0.000000,0.000000,NaN
1,1,36,ladybug_panoramic_000001,372238.874115,55.811356,37.389495,177.252496,178.469506,178.115046,13.246203,2020-11-12 07:23:58,POINT Z (55.81136 37.38949 177.25250),0.500022,0.000085,0.000085,0.263387
2,2,36,ladybug_panoramic_000002,372239.374138,55.811438,37.389522,177.218097,178.502584,178.319568,13.085446,2020-11-12 07:23:59,POINT Z (55.81144 37.38952 177.21810),0.500023,0.000087,0.000087,0.247662
3,3,36,ladybug_panoramic_000003,372239.874160,55.811521,37.389550,177.172916,178.456564,177.944437,12.959536,2020-11-12 07:23:59,POINT Z (55.81152 37.38955 177.17292),0.500022,0.000087,0.000087,0.325289
4,4,36,ladybug_panoramic_000004,372240.374182,55.811604,37.389578,177.127745,178.859275,178.618339,12.458477,2020-11-12 07:24:00,POINT Z (55.81160 37.38958 177.12775),0.500022,0.000088,0.000088,0.325218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,57,36,ladybug_panoramic_000057,372266.875367,55.815910,37.390914,169.079190,178.696118,-179.534623,12.414035,2020-11-12 07:24:26,POINT Z (55.81591 37.39091 169.07919),0.500022,0.000081,0.000081,1.867958
58,58,36,ladybug_panoramic_000058,372267.375389,55.815987,37.390938,168.820853,178.483045,-179.623424,12.25015,2020-11-12 07:24:27,POINT Z (55.81599 37.39094 168.82085),0.500022,0.000081,0.000081,1.859945
59,59,36,ladybug_panoramic_000059,372267.875412,55.816063,37.390962,168.562510,178.559326,-179.505842,11.993371,2020-11-12 07:24:27,POINT Z (55.81606 37.39096 168.56251),0.500023,0.000080,0.000080,1.859984
60,60,36,ladybug_panoramic_000060,372268.375434,55.816140,37.390985,168.304836,178.418475,-179.589118,11.810852,2020-11-12 07:24:28,POINT Z (55.81614 37.39099 168.30484),0.500022,0.000080,0.000080,1.855171


In [11]:
gdf.describe()

,GPS time,X,Y,Z,TimeDiff,PointsDiff,Distance
count,21817.000000,21817.000000,21817.000000,21817.000000,21817.000000,2.181700e+04,2.181700e+04
mean,379141.252290,55.835970,37.447345,174.761741,0.636480,4.968196e-05,4.968196e-05
std,4220.076046,0.023563,0.047051,13.038282,7.745370,4.761015e-04,4.761015e-04
min,372238.374093,55.781439,37.389217,146.231131,0.000000,0.000000e+00,0.000000e+00
25%,375453.291730,55.826173,37.405318,161.312118,0.500022,5.076146e-07,5.076146e-07
50%,379019.777932,55.832533,37.437783,177.541649,0.500022,3.528659e-05,3.528659e-05
75%,383041.222840,55.856697,37.475279,186.107876,0.500023,7.892662e-05,7.892662e-05
max,386124.463166,55.888866,37.573666,200.233964,636.872752,5.954206e-02,5.954206e-02


In [12]:
len(gdf[gdf['GPS time'].diff() > 0.544444])

10

In [ ]:
for i, cur in enumerate(curves):
    folium.plugins.AntPath(np.c_[cur['X'], cur['Y'],cur['Z']], color=colors[i], popup=f'route #{i}', delay=1000).add_to(routing_map)